# Explore manually annotated Curaçao files

## 1. Read files

In [ ]:
import ast
import json
import re
import xml.etree.ElementTree as ET

In [ ]:
data_dir = "../../data/Training_set_V2"

In [ ]:
def get_text_from_file(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()
    return get_text_from_xml(root)

In [ ]:
def get_text_from_xml(root):
    text = ""
    for textline in root.findall(".//{*}TextLine"):
        custom_dict = make_custom_dict(textline.attrib)
        for unicode in textline.findall(".//{*}TextEquiv/{*}Unicode"):
            text += remove_strikethroughs(unicode.text, custom_dict) + "\n"
    return text

In [ ]:
def make_custom_dict(text_line_attributes):
    if "custom" not in text_line_attributes:
        return {}
    custom_tokens = text_line_attributes["custom"].split()
    custom_dict = {}
    while custom_tokens:
        custom_key = custom_tokens.pop(0)
        custom_value = custom_tokens.pop(0)
        while custom_tokens and not re.search("}$", custom_value):
            custom_value += " " + custom_tokens.pop(0)
        if custom_key in custom_dict:
            custom_dict[custom_key].append(ast.literal_eval(json_string_add_quotes(custom_value)))
        else:
            custom_dict[custom_key] = [ast.literal_eval(json_string_add_quotes(custom_value))]
    return custom_dict

In [ ]:
def remove_strikethroughs(text_line, custom_dict):
    if "textStyle" not in custom_dict:
        return text_line
    chars = list(text_line)
    for strikethrough in custom_dict["textStyle"]:
        if "strikethrough" in strikethrough:
            start = int(strikethrough["offset"])
            for i in range(start, start + int(strikethrough["length"])):
                chars[i] = " "
    return "".join(chars)

In [ ]:
def json_string_add_quotes(string):
    return re.sub("{ *", "{ '", 
               re.sub(": *", "': '", 
                   re.sub("; *", "', '",
                       re.sub("} *'", "} ",
                           re.sub("; *}", "' }", string)))))

In [ ]:
def make_file_name(file_id):
    return "p" + str(file_id).zfill(3) + ".xml"

In [ ]:
def read_files():
    texts = []
    for file_id in range(1, 12):
        if file_id != 10:
            texts.append(get_text_from_file(data_dir + "/" + make_file_name(file_id)))
    return texts

In [ ]:
texts = read_files()

## 2. Find entities in texts

In [ ]:
import transformers

transformers.utils.logging.set_verbosity_error()

In [ ]:
def show_names(entities):
    name = ""
    for part in entities:
        if re.search("^B", part["entity"]) and name != "":
            print(name)
            name = ""
        if re.search("(GPE|PERSON)$", part["entity"]):
            if name != "":
                name += " "
            name += part["word"]
    if name != "":
        print(name)  

Tested models (initial number indicates monthly downloads):
* (345) wietsedv/bert-base-dutch-cased-finetuned-conll2002-ner (several false positives)
* (74) Matthijsvanhof/bert-base-dutch-cased-finetuned-NER (not useful, tags everything)
* (16) wietsedv/bert-base-dutch-cased-finetuned-sonar-ner (some false positives)
* (13) proycon/bert-ner-cased-conll2002-nld (did not find any entities)
* (10) proycon/bert-ner-cased-sonar1-nld (found only one entity)
* (10) Matthijsvanhof/bert-base-dutch-cased-finetuned-NER8 (not useful, tags everything)
* (4) [wietsedv/bert-base-dutch-cased-finetuned-udlassy-ner](https://huggingface.co/wietsedv/bert-base-dutch-cased-finetuned-udlassy-ner) (few false positives) **SELECTED**

In [ ]:
run_bert_pipeline = transformers.pipeline(task='ner', model='wietsedv/bert-base-dutch-cased-finetuned-udlassy-ner')

In [ ]:
entities = run_bert_pipeline(texts[1])

In [ ]:
show_names(entities)

## 3. Visualize entities

For list of entity tags of model `wietsedv/bert-base-dutch-cased-finetuned-udlassy-ner`, see [OntoNotes](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf), page 21

In [ ]:
from spacy import displacy

In [ ]:
def convert_entities(entities_in):
    entities_out = []
    for entity in entities_in:
        start_tag = entity["entity"][0]
        label = entity["entity"][2:]
        if start_tag == "B" or not entities_out:
            entities_out.append({"start": entity["start"], "end": entity["end"], "label": label})
        else:
            entities_out[-1]["end"] = entity["end"]
    return entities_out

In [ ]:
def render_text(text, entities):
    displacy.render({ "text": re.sub("\\n", " ", text), 
                      "ents": convert_entities(entities) }, 
                      options = { "colors": { "PERSON": "orange" } }, style = "ent", manual = True)

In [ ]:
render_text(texts[1], entities)

## 4. Post-process entities

Correct entities which end in the middle of a word

In [ ]:
def expand_entities(entities_in, text):
    entities_out = []
    for entity_in in entities_in:
        entity_out = entity_in.copy()
        while (entity_out["end"] < len(text) and 
               (re.search("\w", text[entity_out["end"]]) or re.search("[.,-]", text[entity_out["end"]]))):
            entity_out["word"] += text[entity_out['end']]
            entity_out["end"] += 1
        entities_out.append(entity_out)
    return entities_out

Combine successive entities where the second one has a label starting with I or with the same label type 

In [ ]:
def combine_entities(entities_in):
    entities_out = []
    for entity_in in entities_in:
        entity_out = entity_in.copy()
        if len(entities_out) == 0:
            entities_out.append(entity_out)
        elif re.search("^I-", entity_out["entity"]):
            expand_last_entity(entities_out, entity_out)
        else:
            entity_out["entity"] = re.sub("^[BIE]-", "B-", entity_out["entity"])
            if entity_out["start"] < entities_out[-1]["start"]:
                print("error: entities are not sorted by position!")
            elif entity_out["start"] <= entities_out[-1]["end"] + 1 and entity_out["entity"] == entities_out[-1]["entity"]:
                expand_last_entity(entities_out, entity_out)
            else:
                entities_out.append(entity_out)
    return entities_out

In [ ]:
def expand_last_entity(entities, entity):
    entities[-1]["word"] += " " + entity["word"]
    entities[-1]["end"] = entity["end"]

In [ ]:
entities = run_bert_pipeline(texts[7])

In [ ]:
entities = combine_entities(expand_entities(entities, text))

In [ ]:
render_text(texts[7], entities)

In [ ]:
for text_id in range(0, len(texts)):
    text = texts[text_id]
    entities = run_bert_pipeline(text)
    entities = combine_entities(expand_entities(entities, text))
    print(f"Text {text_id}")
    render_text(text, entities)